# Food-food edges
Similarities based on word2vec embeddings for food description. Also possible for Burpple review text, but not shown here.
> ... terms in the food2vec vocabulary. 

> ... food pairs with sim > 0.95

In [1]:
# python version
import sys
print "system_info: %s"%sys.version
# current working directory
import os
print "path_info: %s"%os.getcwd() 

system_info: 2.7.14 |Anaconda custom (64-bit)| (default, Oct 27 2017, 18:21:12) 
[GCC 7.2.0]
path_info: /home/yueliu/Desktop/workspace_yue/Documentation_201712


In [9]:
import myfunctions as f
f.today()

'20180208'

In [10]:
%%time
loc_decs_ref = dict()
for website in websites:
    file_name = website["_index"]+ "__food__20180102.p"  
    df_retrived = f.retrive_file(file_name)
    loc_decs_ref.update(df_retrived.set_index("loc").to_dict()["_source.description"])

retrived: foodpanda__food__20180102.p
retrived: deliveroo__food__20180102.p
retrived: what_to_eat__food__20180102.p
CPU times: user 5min 22s, sys: 9.5 s, total: 5min 32s
Wall time: 5min 33s


In [12]:
print len(loc_decs_ref)

3069923


In [ ]:
# get desc: loc

In [13]:
file_name = "food_entities__20180208.p"
food_entities = f.retrive_file(file_name)
food_loc_ref = food_entities.set_index("index").to_dict()["locs"]
# loc_food_ref =  {}
# for k,v in food_loc_ref.items():
#     for x in v:
#         loc_food_ref.setdefault(x,[]).append(k)

retrived: food_entities__20180208.p


In [14]:
print len(food_loc_ref)

92298


In [21]:
import pandas as pd
food_descs = pd.DataFrame(food_loc_ref.items(), columns=['index', 'locs']) 
food_descs["desc"] = food_descs["locs"].apply(lambda lst: [loc_decs_ref[l] for l in lst]) 
##removing nan
food_descs["desc"] = food_descs["desc"].apply(lambda lst: [l for l in lst if l==l]) 
print food_descs.shape
food_descs.head(2)

(92298, 3)


,index,locs,desc
0,food_28136,"[foodpanda/menu_item/110873, foodpanda/menu_it...","[Mixed seed parsley garlic butter, Mixed seed ..."
1,food_69958,"[foodpanda/menu_item/106972, deliveroo/food/12...",[]


In [ ]:
%%time
""" from food_descptions, generate embeddings """
from gensim.models import Word2Vec
sentences =  [item for sublist in food_descs["desc"].tolist() for item in sublist]
sentences = [l.split() for l in sentences]
word2vec_filepath = os.path.join(os.getcwd(), 'word2vec_model')
# initiate the model and perform the first epoch of training
food2vec = Word2Vec(sentences, size=100, window=5, min_count=5, sg=1, workers=4)
food2vec.save(word2vec_filepath)
# perform another 11 epochs of training
for i in range(1,12):
    food2vec.train(sentences)
    food2vec.save(word2vec_filepath)
# load the finished model from disk
food2vec = Word2Vec.load(word2vec_filepath)
food2vec.init_sims()

print u'{} training epochs so far.'.format(food2vec.train_count)
print u'{:,} terms in the food2vec vocabulary.'.format(len(food2vec.vocab))

In [ ]:
# build a list of the terms, integer indices,
# and term counts from the food2vec model vocabulary
import pandas as pd
ordered_vocab = [(term, voc.index, voc.count)
                 for term, voc in food2vec.vocab.iteritems()]
# sort by the term counts, so the most common terms appear first
ordered_vocab = sorted(ordered_vocab, key=lambda (term, index, count): -count)
# unzip the terms, integer indices, and counts into separate lists
ordered_terms, term_indices, term_counts = zip(*ordered_vocab)
# create a DataFrame with the food2vec vectors as data,
# and the terms as row labels
word_vectors = pd.DataFrame(food2vec.syn0norm[term_indices, :],
                            index=ordered_terms)
print word_vectors.shape

In [ ]:
def get_related_terms(token, topn=10):
    """
    look up the topn most similar terms to token and print them as a formatted list
    """
    for word, similarity in food2vec.most_similar(positive=[token], 
                                                  topn=topn):
        print u'{:20} {}'.format(word, round(similarity, 3))

In [ ]:
s = "fried"
print s
print get_related_terms(s)
print "\n"
s = "chicken"
print s
print get_related_terms(s)

In [ ]:
%%time
## using weighted average
import numpy as np
vocab = word_vectors.index.tolist()
m = word_vectors.as_matrix()

def find_weight(values, w):
    if w in values.keys():
        return values[w]
    return 0
def desc_embedding(lst):
    words = str(" ".join(lst)).split(" ")
    values = reduce( lambda d, c: d.update([(c, d.get(c,0)+1)]) or d, words, {})
    weights = [find_weight(values, w) for w in vocab]
    try:
        return np.average(m, axis = 0, weights = weights)
    except:
        """ ZeroDivisionError: Weights sum to zero, can't be normalized """
        return np.average(m, axis = 0)

In [ ]:
%%time
food_descs["text_vector"] = food_descs["desc"].apply(desc_embedding)

In [ ]:
text_vectors = food_descs.set_index("index").to_dict()["text_vector"]

def compute_sim(v1, v2):
    from sklearn.metrics.pairwise import cosine_similarity
    return cosine_similarity([v1], [v2])[0][0]
def find_most_similar(v1, topn=10):
    sim_lst = []
    for v in names_vectors.keys():
        sim_lst.append((v, compute_sim(names_vectors[v1], names_vectors[v])))
    sim_lst.sort(key=lambda tup: tup[1], reverse=True) 
    return sim_lst[1:topn+1]

In [ ]:
## find all sim
from sklearn.metrics.pairwise import cosine_similarity
lst = food_descs["text_vector"].tolist()
sims = cosine_similarity(lst)
## convert to df
import pandas as pd
df_sims = pd.DataFrame(sims, index=food_descs["index"], columns=food_descs["index"])

In [ ]:
treshold = 0.95
high_sim = []
names = food_descs["index"].tolist()
for i in range(len(lst)):
    for k in range(i+1, len(lst)):
        if sims[i][k]> treshold and sims[i][k]<0.999999999999999:
            high_sim.append((names[i], names[k], sims[i][k]))  
print len(high_sim)

In [ ]:
# sample
high_sim[:5]

In [ ]:
ff_desc_sim = []
relation_type = "ff_desc_sim"
index = 0
for triplet in high_sim:
    ff_desc_sim.append([index, triplet[0], triplet[2], triplet[1], relation_type])
    index+=1
    ff_desc_sim.append([index, triplet[1], triplet[2], triplet[0], relation_type])
    index+=1
print "number of ff_desc_sim relations found: %d"%index

In [ ]:
file_name = "__".join(["ff_desc_sim", f.today()]) + ".p"
f.save_file(ff_desc_sim, file_name)